In [ ]:
streamlit==1.31.0
pandas>=2.0.0
numpy>=1.24.0
plotly>=5.18.0
scikit-learn>=1.3.0
yfinance>=0.2.36
# tensorflow-quantum  <-- (Uncomment this only if you are ready for a huge build size)

In [ ]:
!pip install streamlit==1.31.0 pandas>=2.0.0 numpy>=1.24.0 plotly>=5.18.0 scikit-learn>=1.3.0 yfinance>=0.2.36 pandas_datareader polygon-api-client
import streamlit as st
import time

# Page Configuration
st.set_page_config(
    page_title="My Project Interface",
    page_icon="⚡",
    layout="centered"
)

# Header
st.title("Project Dashboard")
st.markdown("---")

# Sidebar for inputs
with st.sidebar:
    st.header("Configuration")
    input_mode = st.selectbox("Select Mode", ["Standard", "Advanced", "Debug"])
    user_param = st.slider("Parameter Tuning", 0, 100, 50)

    st.info("Adjust settings above before running the process.")

# Main Interface
col1, col2 = st.columns(2)

with col1:
    st.subheader("Input Data")
    text_input = st.text_area("Enter your data or query here:", height=150)

with col2:
    st.subheader("System Status")
    status_placeholder = st.empty()
    status_placeholder.success("System Ready")

# Action Button
if st.button("Run Process", type="primary", use_container_width=True):
    if not text_input:
        st.warning("Please provide input data first.")
    else:
        # ---------------------------------------------------------
        # YOUR LOGIC GOES HERE
        # Example: result = my_function(text_input, user_param)
        # ---------------------------------------------------------

        status_placeholder.info("Processing...")
        progress_bar = st.progress(0)

        # Simulating work
        for i in range(100):
            time.sleep(0.01)
            progress_bar.progress(i + 1)

        status_placeholder.success("Complete!")

        # Display Results
        st.markdown("### Results")
        st.success(f"Processed: {text_input[:20]}... with Parameter: {user_param}")
        st.json({"status": "success", "mode": input_mode, "output_value": 42})

# Footer
st.markdown("---")
st.caption("Deployed via GitHub & Railway")

In [ ]:
import os, time, random, threading, shelve
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
import pandas as pd
import requests
import yfinance as yf
from polygon import RESTClient
import torch
import matplotlib.pyplot as plt

# Quick env setup examples (run in a separate cell or uncomment here)
#%env ALPHAVANTAGE_KEY=your_alpha_key_here
#%env SEC_USER_AGENT=youremail@example.com

ALPHAVANTAGE_KEY = os.getenv("ALPHAVANTAGE_KEY", "")
SEC_USER_AGENT = os.getenv("SEC_USER_AGENT", "your_email@example.com")

# Config
CACHE_FILE = "market_fetch_cache.db"   # shelve file (disk cache)
CACHE_TTL = 300                        # seconds
PROVIDER_MIN_INTERVALS = {
    "yfinance": 0.2,
    "stooq": 0.5,
    "alpha": 12.0,
    "yahoo_json": 0.5,
    "sec": 0.5,
    "polygon": 0.5
}
MAX_WORKERS = 6

# Simulation defaults
derivative_per_ticker = 3
eta = 0.4
phi_ladder_vals = [0.3, 0.8, 1.618]
update_interval = 0.1
max_steps = 200
cross_sample_k = 10
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Advanced Market Structure Parameters
ENABLE_BASEL_CONSTRAINTS = True
MAX_RISK_RATIO = 0.3  # max % of capital per agent in risky assets
DELEVERAGING_SEVERITY = 0.02  # how aggressively overexposed agents deleverage

ENABLE_TIERED_LIQUIDITY = True
SLOW_AGENT_RATIO = 0.3  # fraction of agents that are "slow" (institutional)
SLOW_REACTION_RATE = 0.05  # how slowly slow agents respond
SLOW_AGENT_Z3_WEIGHT = 2.0  # institutional influence on consensus

ENABLE_DELIVERY_FRICTION = True
COMMODITY_TICKERS = ['GLD', 'SLV', 'GC=F', 'SI=F']  # add your metal tickers
MAX_DELIVERY_RATE = 0.02  # max % of volume that can be physically delivered per step
SCARCITY_PREMIUM_FACTOR = 0.1  # price boost when physical is scarce

ENABLE_REGIME_SHOCKS = True
SHOCK_THRESHOLD = 0.10  # trigger shock when cluster diverges >10% from baseline
SHOCK_MAGNITUDE = 0.15  # size of geopolitical shock
LATAM_ENERGY_TICKERS = ['PBR', 'EC', 'VALE']  # example cluster

ENABLE_MEMORY = True
MEMORY_ALPHA = 0.1  # weight of new info vs history (lower = more memory)
MEMORY_INFLUENCE = 0.05  # how much memory pulls prices

ENABLE_INFLUENCE_WEIGHTING = True
INFLUENCE_POWER_SCALE = 10.0  # amplification factor for big money

# Disk cache (shelve) with TTL
_cache_lock = threading.Lock()

def cache_get(key):
    with _cache_lock:
        with shelve.open(CACHE_FILE) as db:
            if key in db:
                ts, val = db[key]
                if time.time() - ts < CACHE_TTL:
                    return val
                else:
                    del db[key]
            return None

def cache_set(key, val):
    with _cache_lock:
        with shelve.open(CACHE_FILE) as db:
            db[key] = (time.time(), val)

# Per-provider rate limiter
_provider_locks = {name: threading.Lock() for name in PROVIDER_MIN_INTERVALS}
_provider_last_call = {name: 0.0 for name in PROVIDER_MIN_INTERVALS}

def rate_limit(provider_name):
    min_interval = PROVIDER_MIN_INTERVALS.get(provider_name, 0.5)
    lock = _provider_locks[provider_name]
    with lock:
        now = time.time()
        elapsed = now - _provider_last_call[provider_name]
        wait = max(0.0, min_interval - elapsed)
        if wait > 0:
            time.sleep(wait)
        _provider_last_call[provider_name] = time.time()

# Backoff retry helper
def backoff_retry(fn, max_attempts=4, base_delay=0.5):
    last_exc = None
    for attempt in range(1, max_attempts + 1):
        try:
            return fn()
        except Exception as e:
            last_exc = e
            if attempt == max_attempts:
                raise
            sleep = base_delay * (2 ** (attempt - 1)) * (0.8 + 0.4 * random.random())
            time.sleep(sleep)
    raise last_exc

# Providers (price-focused)
def provider_yfinance_quote(ticker):
    key = f"yf_quote:{ticker}"
    cached = cache_get(key)
    if cached is not None:
        return cached
    def _call():
        rate_limit("yfinance")
        t = yf.Ticker(ticker)
        hist = t.history(period="1d", interval="1d")
        if hist is None or hist.empty:
            raise RuntimeError("yfinance empty")
        latest = hist.iloc[-1]
        out = {"provider": "yfinance", "ticker": ticker, "price": float(latest["Close"]), "volume": int(latest.get("Volume", 0) or 0)}
        cache_set(key, out)
        return out
    return backoff_retry(_call, max_attempts=3, base_delay=0.3)

def provider_stooq_history(ticker):
    key = f"stooq:{ticker}"
    cached = cache_get(key)
    if cached is not None:
        return cached
    def _call():
        rate_limit("stooq")
        try:
            import pandas_datareader.data as web
            end = datetime.now()
            start = end - timedelta(days=7)
            df = web.DataReader(ticker.upper(), 'stooq', start=start, end=end)
            if df is None or df.empty:
                raise RuntimeError("stooq empty")
            row = df.iloc[0]
            out = {"provider": "stooq", "ticker": ticker, "price": float(row["Close"]), "volume": int(row.get("Volume", 0) or 0)}
            cache_set(key, out)
            return out
        except Exception:
            url = f"https://stooq.com/q/d/l/?s={ticker.lower()}&i=d"
            r = requests.get(url, timeout=8)
            r.raise_for_status()
            df = pd.read_csv(pd.compat.StringIO(r.text))
            if df.empty:
                raise RuntimeError("stooq csv empty")
            latest = df.iloc[-1]
            out = {"provider": "stooq", "ticker": ticker, "price": float(latest["Close"]), "volume": int(latest.get("Volume", 0) or 0)}
            cache_set(key, out)
            return out
    return backoff_retry(_call, max_attempts=3, base_delay=0.5)

def provider_alpha_vantage_quote(ticker):
    if not ALPHAVANTAGE_KEY:
        raise RuntimeError("Alpha Vantage key not set")
    key = f"alpha:{ticker}"
    cached = cache_get(key)
    if cached is not None:
        return cached
    def _call():
        rate_limit("alpha")
        url = "https://www.alphavantage.co/query"
        params = {"function": "GLOBAL_QUOTE", "symbol": ticker, "apikey": ALPHAVANTAGE_KEY}
        r = requests.get(url, params=params, timeout=12)
        r.raise_for_status()
        data = r.json()
        if "Global Quote" not in data or not data["Global Quote"]:
            raise RuntimeError("alpha empty")
        g = data["Global Quote"]
        price = float(g.get("05. price", 0.0))
        vol = int(float(g.get("06. volume", 0) or 0))
        out = {"provider": "alpha", "ticker": ticker, "price": price, "volume": vol}
        cache_set(key, out)
        return out
    return backoff_retry(_call, max_attempts=3, base_delay=1.0)

def provider_yahoo_json(ticker):
    key = f"yahoo_json:{ticker}"
    cached = cache_get(key)
    if cached is not None:
        return cached
    def _call():
        rate_limit("yahoo_json")
        url = f"https://query1.finance.yahoo.com/v7/finance/quote?symbols={ticker}"
        r = requests.get(url, timeout=8)
        r.raise_for_status()
        j = r.json()
        q = j.get("quoteResponse", {}).get("result", [])
        if not q:
            raise RuntimeError("yahoo json empty")
        q0 = q[0]
        out = {"provider": "yahoo_json", "ticker": ticker, "price": float(q0.get("regularMarketPrice", 0.0)), "volume": int(q0.get("regularMarketVolume", 0) or 0)}
        cache_set(key, out)
        return out
    return backoff_retry(_call, max_attempts=2, base_delay=0.5)

def provider_polygon_quote(ticker):
    key = f"polygon:{ticker}"
    cached = cache_get(key)
    if cached is not None:
        return cached
    def _call():
        rate_limit("polygon")
        client = RESTClient()
        agg = client.get_previous_close_agg(ticker)
        if not agg or not agg.results:
            raise RuntimeError("polygon empty")
        latest = agg.results[0]
        out = {"provider": "polygon", "ticker": ticker, "price": latest['c'], "volume": latest['v']}
        cache_set(key, out)
        return out
    return backoff_retry(_call, max_attempts=3, base_delay=0.5)

# SEC EDGAR helpers (filings/facts)
SEC_HEADERS = {"User-Agent": SEC_USER_AGENT}

def fetch_sec_ticker_cik_map():
    key = "sec_ticker_cik_map"
    cached = cache_get(key)
    if cached is not None:
        return cached
    rate_limit("sec")
    url = "https://www.sec.gov/files/company_tickers.json"
    r = requests.get(url, headers=SEC_HEADERS, timeout=12)
    r.raise_for_status()
    data = r.json()
    mapping = {v["ticker"].upper(): str(v["cik_str"]).zfill(10) for v in data.values()}
    cache_set(key, mapping)
    return mapping

def provider_sec_submissions(ticker):
    key = f"sec_subs:{ticker}"
    cached = cache_get(key)
    if cached is not None:
        return cached
    mapping = fetch_sec_ticker_cik_map()
    ticker_up = ticker.upper()
    if ticker_up not in mapping:
        raise RuntimeError("CIK not found for ticker")
    cik = mapping[ticker_up]
    rate_limit("sec")
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    r = requests.get(url, headers=SEC_HEADERS, timeout=12)
    r.raise_for_status()
    subs = r.json()
    recent = subs.get("filings", {}).get("recent", {})
    out = {"provider": "sec", "ticker": ticker, "recent_forms": recent.get("form", [])[:10], "recent_dates": recent.get("filingDate", [])[:10]}
    cache_set(key, out)
    return out

def provider_sec_company_facts(ticker):
    key = f"sec_facts:{ticker}"
    cached = cache_get(key)
    if cached is not None:
        return cached
    mapping = fetch_sec_ticker_cik_map()
    ticker_up = ticker.upper()
    if ticker_up not in mapping:
        raise RuntimeError("CIK not found for ticker")
    cik = mapping[ticker_up]
    rate_limit("sec")
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    r = requests.get(url, headers=SEC_HEADERS, timeout=12)
    r.raise_for_status()
    facts = r.json()
    usgaap = facts.get("facts", {}).get("us-gaap", {})
    assets = None
    for key_name in ("Assets", "AssetsCurrent", "AssetsNoncurrent"):
        if key_name in usgaap:
            arr = usgaap[key_name].get("units", {}).get("USD", [])
            if arr:
                assets = arr[-1].get("val")
                break
    out = {"provider": "sec", "ticker": ticker, "assets": assets}
    cache_set(key, out)
    return out

# Provider priority list and manager
PROVIDERS = [
    provider_yfinance_quote,
    provider_stooq_history,
    provider_alpha_vantage_quote if ALPHAVANTAGE_KEY else None,
    provider_yahoo_json,
    provider_polygon_quote
]

# filter None
PROVIDERS = [p for p in PROVIDERS if p is not None]

def fetch_with_fallback(ticker):
    last_exc = None
    for prov in PROVIDERS:
        try:
            return prov(ticker)
        except Exception as e:
            last_exc = e
            continue
    # price providers failed; return SEC fallback (filings/facts) if available
    try:
        sec_info = provider_sec_submissions(ticker)
        return {"provider": "sec_fallback", "ticker": ticker, "note": "price unavailable", "sec": sec_info}
    except Exception:
        pass
    raise last_exc

def bulk_fetch(tickers, max_workers=MAX_WORKERS):
    results = {}
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {ex.submit(fetch_with_fallback, t): t for t in tickers}
        for fut in as_completed(futures):
            t = futures[fut]
            try:
                results[t] = fut.result()
            except Exception as e:
                results[t] = {"error": str(e)}
    return results

# Tickers: create fallback CSV if missing and load
def ensure_sp500_csv():
    sp500_tickers = pd.DataFrame({
        'Symbol': [
            'AAPL','MSFT','GOOGL','AMZN','NVDA','META','TSLA','AVGO',
            'COST','NFLX','AMD','LIN','ADBE','ACN','TMUS','ABNB','QCOM',
            'GLD', 'SLV', 'PBR', 'EC', 'VALE'
        ]
    })
    sp500_tickers.to_csv('sp500_tickers.csv', index=False)

ensure_sp500_csv()

def load_tickers(txt_path='/content/tickers.txt', csv_path='sp500_tickers.csv'):
    if os.path.exists(txt_path):
        with open(txt_path, 'r') as f:
            tickers = [line.strip() for line in f if line.strip()]
    else:
        tickers = pd.read_csv(csv_path)['Symbol'].astype(str).str.strip().tolist()
    tickers = ['XYZ' if t == 'SQ' else t for t in tickers]
    return tickers

txt_tickers = load_tickers()
print("Tickers to fetch (sample):", txt_tickers[:10])

# Fetch initial market data using bulk_fetch
print("Fetching initial market data (this may take a moment)...")
raw_results = bulk_fetch(txt_tickers)

# Build market_data DataFrame from successful price results
rows = []
for t, res in raw_results.items():
    if res is None:
        continue
    if "error" in res:
        continue
    # price providers return price & volume; sec_fallback returns sec info
    if res.get("provider") == "sec_fallback":
        # skip as price source but keep as metadata if needed
        continue
    price = res.get("price")
    vol = res.get("volume", 0)
    if price is None:
        continue
    rows.append({
        'security_id': t,
        'underlying_id': t,
        'price': float(price),
        'volume': int(vol),
        'sentiment': 0.0,
        'provider': res.get("provider")
    })

market_data = pd.DataFrame(rows)
if market_data.empty:
    raise RuntimeError("Failed to fetch market data from all providers. Check keys and network.")

print(f"Fetched price data for {len(market_data)} tickers.")

# Create derivatives
derivs = []
for _, row in market_data.iterrows():
    for d in range(derivative_per_ticker):
        deriv = row.copy()
        deriv['security_id'] = f"{row['security_id']}_D{d+1}"
        deriv['price'] = float(row['price']) * float(np.random.uniform(0.95, 1.05))
        deriv['volume'] = int(row['volume'] * np.random.uniform(0.1, 0.5))
        deriv['sentiment'] = 0.0
        derivs.append(deriv)
if derivs:
    market_data = pd.concat([market_data, pd.DataFrame(derivs)], ignore_index=True)

# Convert to tensors safely
if market_data.empty:
    raise ValueError("market_data is empty after fetching. No valid stock data.")

securities = market_data['security_id'].astype(str).tolist()
num_agents = len(securities)
print(f"Total agents (including derivatives): {num_agents}")

# Fix: Create agent_prices AND baseline_prices in one shot from the same data
price_array = market_data['price'].values
agent_prices = torch.tensor(price_array, dtype=torch.float32, device=device)

# IMMEDIATE CHECK
print(f"🚨 agent_prices[19] RIGHT AFTER CREATION = ${agent_prices[19].item():.2f}")
print(f"🚨 Expected (from market_data) = ${market_data.iloc[19]['price']:.2f}")
assert abs(agent_prices[19].item() - market_data.iloc[19]['price']) < 0.01, "Prices don't match!"

baseline_prices = torch.tensor(price_array, dtype=torch.float32, device=device)  # Independent copy, not .clone()

# Verify they're identical
print(f"Prices equal after creation: {torch.allclose(agent_prices, baseline_prices)}")
assert torch.allclose(agent_prices, baseline_prices), "agent_prices and baseline_prices don't match at creation!"

agent_volumes = torch.tensor(market_data['volume'].values, dtype=torch.float32, device=device)
agent_sentiment = torch.tensor(market_data['sentiment'].values, dtype=torch.float32, device=device)
phi_agents = 1.0 + agent_volumes / (agent_volumes.max() + 1e-9)
phi_ladder = torch.tensor(phi_ladder_vals, device=device)

# Cluster mapping
cluster_map = {}
for idx, row in market_data.reset_index().iterrows():
    sec = row['security_id']
    if '_D' in sec:
        parent = sec.rsplit('_D', 1)[0]
    else:
        parent = row['underlying_id'] if pd.notna(row['underlying_id']) else sec
    cluster_map.setdefault(parent, []).append(idx)

cluster_indices = {k: torch.tensor(v, dtype=torch.long, device=device) for k, v in cluster_map.items()}

# Initialize advanced state tensors
# Capital tracking for Basel constraints
agent_capital = agent_volumes.clone() * agent_prices.clone()  # initial capital = volume * price

# Agent type classification (slow vs fast)
num_slow = int(num_agents * SLOW_AGENT_RATIO)
is_slow_agent = torch.zeros(num_agents, dtype=torch.bool, device=device)
# Assign largest volume agents as "slow" (institutional)
slow_indices = torch.topk(agent_volumes, num_slow).indices
is_slow_agent[slow_indices] = True

# Physical inventory for delivery-constrained assets
physical_inventory = torch.ones(num_agents, dtype=torch.float32, device=device) * 1000.0  # arbitrary starting inventory
is_commodity = torch.zeros(num_agents, dtype=torch.bool, device=device)
for idx, sec in enumerate(securities):
    underlying = market_data.iloc[idx]['underlying_id'] if '_D' not in sec else sec.rsplit('_D', 1)[0]
    if any(ticker in underlying for ticker in COMMODITY_TICKERS):
        is_commodity[idx] = True

# Memory state
agent_memory = agent_prices.clone()

# Shock tracking
shock_applied = torch.zeros(num_agents, dtype=torch.bool, device=device)

print(f"Agent classification: {is_slow_agent.sum().item()} slow (institutional), {(~is_slow_agent).sum().item()} fast (retail)")
print(f"Commodity agents: {is_commodity.sum().item()}")

# Market Structure Tweak Functions
def apply_basel_constraints(prices, volumes, capital):
    """1. Basel III capital constraints - force deleveraging if overexposed"""
    if not ENABLE_BASEL_CONSTRAINTS:
        return prices

    risk_exposure = prices * volumes
    max_allowed = MAX_RISK_RATIO * capital
    excess = (risk_exposure - max_allowed).clamp(min=0)

    overexposed = excess > 0
    if overexposed.any():
        deleveraging_penalty = DELEVERAGING_SEVERITY * (excess / (capital + 1e-9))
        prices = prices * (1 - deleveraging_penalty)

    return prices

def apply_tiered_liquidity(prices, target_prices, is_slow):
    """2. Slow agents (institutions) respond with delay"""
    if not ENABLE_TIERED_LIQUIDITY:
        return prices

    # Fast agents update normally, slow agents move partially toward target
    delta = target_prices - prices
    slow_delta = SLOW_REACTION_RATE * delta
    fast_delta = delta

    prices = torch.where(is_slow, prices + slow_delta, prices + fast_delta)
    return prices

def apply_delivery_friction(prices, volumes, inventory, is_commodity_agent):
    """3. Physical delivery constraints for commodities"""
    if not ENABLE_DELIVERY_FRICTION:
        return prices, inventory

    # Attempt to deliver based on volume demand
    delivery_attempt = volumes * MAX_DELIVERY_RATE

    # Deplete inventory
    inventory = inventory - delivery_attempt
    inventory = inventory.clamp(min=0)

    # Scarcity premium when inventory runs low
    scarcity = 1.0 / (inventory + 1.0)  # inverse of available supply
    premium = SCARCITY_PREMIUM_FACTOR * scarcity

    # Only apply to commodity agents
    prices = torch.where(is_commodity_agent, prices * (1 + premium), prices)

    return prices, inventory

def apply_regime_shock(prices, baseline, cluster_indices, step):
    """4. Endogenous shocks when clusters diverge too much"""
    if not ENABLE_REGIME_SHOCKS:
        return prices

    # Check for LATAM energy cluster stress
    latam_idxs = []
    for idx, sec in enumerate(securities):
        underlying = market_data.iloc[idx]['underlying_id'] if '_D' not in sec else sec.rsplit('_D', 1)[0]
        if any(ticker in underlying for ticker in LATAM_ENERGY_TICKERS):
            latam_idxs.append(idx)

    if len(latam_idxs) > 0:
        latam_tensor = torch.tensor(latam_idxs, device=device)
        divergence = (prices[latam_tensor] - baseline[latam_tensor]).abs().mean()

        if divergence > SHOCK_THRESHOLD and not shock_applied[latam_tensor[0]]:
            # Apply one-time shock
            shock = torch.randn(len(latam_idxs), device=device) * SHOCK_MAGNITUDE
            prices[latam_tensor] = prices[latam_tensor] * (1 + shock)
            shock_applied[latam_tensor] = True
            print(f"  ⚠️  REGIME SHOCK applied to LATAM energy cluster at step {step} (divergence: {divergence:.2%})")

    return prices

def apply_memory_feedback(prices, memory, alpha, influence):
    """5. Exponential moving average memory creates support/resistance"""
    if not ENABLE_MEMORY:
        return prices, memory

    # Update memory: EMA of prices
    memory = alpha * prices + (1 - alpha) * memory

    # Pull prices slightly toward memory (support/resistance effect)
    prices = prices + influence * (memory - prices)

    return prices, memory

def compute_influence_weighted_consensus(prices, volumes, is_slow):
    """6. Dollar-weighted agent influence on global consensus"""
    if not ENABLE_INFLUENCE_WEIGHTING:
        return prices.mean().item()

    # Dollar-weighted power
    agent_power = (volumes * prices) / ((volumes * prices).sum() + 1e-9)

    # Slow agents get extra weight in consensus
    z3_weights = torch.where(is_slow,
                             agent_power * SLOW_AGENT_Z3_WEIGHT,
                             agent_power)

    global_z3 = (prices * z3_weights).sum() / (z3_weights.sum() + 1e-9)
    return global_z3.item()

# Enhanced Simulation Loop
plt.ion()
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

history_global = []
top_k = min(20, num_agents)
history_top_agents = torch.zeros((max_steps, top_k), device=device)

prev_agent_prices = agent_prices.clone()
top_indices = torch.topk(agent_volumes, top_k).indices

for step in range(max_steps):
    ladder_idx = step % len(phi_ladder)
    phi_step = 0.2 * phi_agents * phi_ladder[ladder_idx]

    # Update sentiment (TODO: Integrate real X sentiment fetch here)
    base_sent = 0.0  # Placeholder; replace with actual sentiment calculation
    agent_sentiment = torch.full((num_agents,), base_sent, device=device, dtype=torch.float32)
    agent_sentiment = torch.clamp(agent_sentiment, -0.5, 0.5)
    phi_step = phi_step * (1.0 + 0.3 * agent_sentiment)
    phi_step = torch.clamp(phi_step, 0.0, 1.0)

    # Cross-agent sampling per cluster
    sampled_mean = torch.empty_like(agent_prices)
    for parent, idxs in cluster_indices.items():
        if idxs.numel() == 0:
            continue
        k = idxs.numel()
        rand_idx = torch.randint(0, k, (k,), device=device)
        local_sample = idxs[rand_idx]
        local_mean = agent_prices[local_sample].view(k).mean()
        sampled_mean[idxs] = local_mean

    # Compute global consensus with influence weighting
    global_state = compute_influence_weighted_consensus(agent_prices, agent_volumes, is_slow_agent)
    sampled_mean = 0.9 * sampled_mean + 0.1 * global_state

    # Base price update (target prices)
    noise = eta * torch.randn_like(agent_prices)
    momentum = 0.01 * (agent_prices - torch.roll(agent_prices, 1))
    target_prices = agent_prices + phi_step * (sampled_mean - agent_prices) + noise + momentum

    # APPLY TWEAKS IN ORDER
    # Tweak 2: Tiered liquidity (slow vs fast agents)
    agent_prices = apply_tiered_liquidity(agent_prices, target_prices, is_slow_agent)

    # Tweak 5: Memory feedback
    agent_prices, agent_memory = apply_memory_feedback(agent_prices, agent_memory, MEMORY_ALPHA, MEMORY_INFLUENCE)

    # Tweak 1: Basel III constraints
    agent_prices = apply_basel_constraints(agent_prices, agent_volumes, agent_capital)

    # Tweak 3: Delivery friction
    agent_prices, physical_inventory = apply_delivery_friction(agent_prices, agent_volumes, physical_inventory, is_commodity)

    # Tweak 4: Regime shocks
    agent_prices = apply_regime_shock(agent_prices, baseline_prices, cluster_indices, step)

    # Weak pull toward baseline (original reversion)
    revert_strength = 0.01
    agent_prices = agent_prices + revert_strength * (baseline_prices - agent_prices)

    # Safety checks
    if torch.isnan(agent_prices).any() or torch.isinf(agent_prices).any():
        print("Numerical explosion, stopping at step", step)
        break
    if agent_prices.abs().max().item() > 1e6:
        print("Price magnitude too large, stopping at step", step)
        break

    # Store history
    history_global.append(global_state)
    history_top_agents[step, :top_k] = agent_prices[top_indices].cpu()

    # Trend signals
    rel_move = (agent_prices - prev_agent_prices) / (prev_agent_prices.abs() + 1e-9)
    signals = {}
    for i, sec in enumerate(securities):
        move = rel_move[i].item()
        if move > 0.01:
            signals[sec] = 'BUY'
        elif move < -0.01:
            signals[sec] = 'SELL'
        else:
            signals[sec] = 'HOLD'

    # Plotting every 10 steps
    if (step + 1) % 10 == 0 or step == max_steps - 1:
        ax1.clear(); ax2.clear()
        ax1.plot(history_global, color='blue', linewidth=2, label='Global Z3 (Influence-Weighted)')
        ax1.set_title("Global Z3 Consensus + Multi-Layer Market Structure")
        ax1.set_ylabel("Global Z3 ($)")
        ax1.legend(); ax1.grid(True, alpha=0.3)

        for i in range(history_top_agents.shape[1]):
            ax2.plot(history_top_agents[:step+1, i].cpu().numpy(), alpha=0.7)
        ax2.set_title("Top Influencer Agents")
        ax2.set_xlabel("Step"); ax2.set_ylabel("Agent Price ($)")
        ax2.grid(True, alpha=0.3)
        plt.pause(0.01)

        active = {k: v for k, v in signals.items() if v != 'HOLD'}
        print(f"\nStep {step+1}/{max_steps} | Global Z3: ${global_state:.2f}")

        # Show commodity inventory levels if enabled
        if ENABLE_DELIVERY_FRICTION and is_commodity.any():
            avg_commodity_inventory = physical_inventory[is_commodity].mean().item()
            print(f"  📦 Avg Commodity Inventory: {avg_commodity_inventory:.0f} units")

        for k, v in list(active.items())[:20]:
            print(f"  {k}: {v}")
        print(f"  ... {len(active)} active signals out of {len(signals)} agents")

    prev_agent_prices = agent_prices.clone()
    time.sleep(update_interval)

print("\n✅ Simulation finished.")

# Summary output
sample_providers = market_data[['security_id','provider']].drop_duplicates().head(10).to_dict(orient='records')
print(f"Agents: {num_agents}; sample providers for fetched tickers:")
for r in sample_providers:
    print(r)

In [ ]:

print("\n🔍 LATAM CLUSTER DIVERGENCE DETAIL")
print("="*60)
latam_tickers = ['PBR', 'EC', 'VALE']
for ticker in latam_tickers:
    if ticker in securities:  # Check securities list, not market_data
        idx = securities.index(ticker)  # Get tensor index from securities list
        current = agent_prices[idx].item()
        baseline = baseline_prices[idx].item()
        divergence = ((current - baseline) / baseline) * 100
        print(f"{ticker:6s} | Current: ${current:7.2f} | Baseline: ${baseline:7.2f} | Divergence: {divergence:+7.2f}%")

In [ ]:
# Right after: baseline_prices = agent_prices.clone().detach()
# BEFORE any other code

print("\n🚨 IMMEDIATE TENSOR CHECK (right after creation)")
print("="*60)
for ticker in ['PBR', 'EC', 'VALE']:
    if ticker in securities:
        idx = securities.index(ticker)
        print(f"{ticker} | agent_prices[{idx}]=${agent_prices[idx].item():.2f} | baseline_prices[{idx}]=${baseline_prices[idx].item():.2f}")

print(f"\nTensor means: agent={agent_prices.mean().item():.2f} | baseline={baseline_prices.mean().item():.2f}")
print(f"Are they equal? {torch.allclose(agent_prices, baseline_prices)}")